In [15]:
import pandas as pd

In [16]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/1/School Characteristics.csv')

/tmp/ipython-input-312515032.py:1: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/1/School Characteristics.csv')


In [23]:

df_filtered = df[
    (df['LEA_STATE_NAME'] == 'CALIFORNIA') &
    (df['SCH_STATUS_CHARTER'] == 'Yes')
]
columns_to_keep = ['LEAID', 'LEA_NAME', 'SCHID', 'SCH_NAME', 'COMBOKEY']
df_charter = df_filtered[columns_to_keep]

print(df_charter.head())
print(df_charter.shape)

       LEAID           LEA_NAME  SCHID                              SCH_NAME  \
5047  600011  Fort Sage Unified  12763                    Mt. Lassen Charter   
5276  600034    Windsor Unified   6983        Cali Calmecac Language Academy   
5297  600036    Natomas Unified  11087                      Westlake Charter   
5299  600036    Natomas Unified  11735         Natomas Pacific Pathways Prep   
5301  600036    Natomas Unified  12523  Natomas Pacific Pathways Prep Middle   

         COMBOKEY  
5047  60001112763  
5276  60003406983  
5297  60003611087  
5299  60003611735  
5301  60003612523  
(1271, 5)


In [60]:
edge = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/1/EDGE_GEOCODE_PUBLICSCH_2122.xlsx', dtype=str)

edge_ca = edge[edge['STATE'].str.strip().str.upper() == 'CA']

# 2) Normalize keys: drop leading zeros
edge_ca = edge_ca.copy()
edge_ca['NCESSCH_norm'] = edge_ca['NCESSCH'].str.strip().str.lstrip('0')

df_charter = df_charter.copy()
df_charter['COMBOKEY_norm'] = (
    df_charter['COMBOKEY'].astype(str).str.strip().str.lstrip('0')
)

# 3) Merge on COMBOKEY <-> NCESSCH
merged = df_charter.merge(
    edge_ca[['NCESSCH','NCESSCH_norm','NMCNTY','CITY','LOCALE','LAT','LON']],
    left_on='COMBOKEY_norm',
    right_on='NCESSCH_norm',
    how='left'
)

# 4) Keep columns
out_cols = ['LEAID','LEA_NAME','SCHID','SCH_NAME','CITY','COMBOKEY','NMCNTY','LOCALE','LAT','LON']
out = merged[out_cols]

print(f"Rows in df_charter: {len(df_charter)}")
print(f"Rows matched with geocodes: {out['LAT'].notna().sum()}")
print(out.head())

out_path = '/content/drive/MyDrive/Colab Notebooks/1/CA_charter_with_geo.csv'
out.to_csv(out_path, index=False)
print(f"Saved: {out_path}")

Rows in df_charter: 1271
Rows matched with geocodes: 1266
    LEAID           LEA_NAME  SCHID                              SCH_NAME  \
0  600011  Fort Sage Unified  12763                    Mt. Lassen Charter   
1  600034    Windsor Unified   6983        Cali Calmecac Language Academy   
2  600036    Natomas Unified  11087                      Westlake Charter   
3  600036    Natomas Unified  11735         Natomas Pacific Pathways Prep   
4  600036    Natomas Unified  12523  Natomas Pacific Pathways Prep Middle   

         CITY     COMBOKEY             NMCNTY LOCALE        LAT          LON  
0     Herlong  60001112763      Lassen County     33    40.4211  -120.650932  
1     Windsor  60003406983      Sonoma County     21  38.550242   -122.82712  
2  Sacramento  60003611087  Sacramento County     11   38.67564  -121.526258  
3  Sacramento  60003611735  Sacramento County     11    38.6551  -121.546082  
4  Sacramento  60003612523  Sacramento County     11    38.6551  -121.546082  
Saved

In [61]:
print(out['CITY'].astype(str).str.strip().str.casefold().value_counts())


CITY
los angeles    175
san jose        52
san diego       49
oakland         38
sacramento      32
              ... 
beale afb        1
pittsburg        1
yerington        1
san rafael       1
freedom          1
Name: count, Length: 331, dtype: int64
